In [3]:
import numpy as np
import matplotlib.pyplot as plt
import gym
from gym.envs.registration import register
import time
import os
from .main import egas, computeQValue, decay, punish


ImportError: attempted relative import with no known parent package

In [4]:
env = gym.make('CartPole-v1', render_mode="human")
env.reset()

(array([-0.0383417 ,  0.01542443, -0.03563764, -0.03051744], dtype=float32),
 {})

In [5]:
env.reset()
for _ in range(100):
    env.render()
    observation, reward, done, info, check = env.step(env.action_space.sample())
    time.sleep(0.03)
    if done:
        env.reset()
env.close()

C:\Users\bherw\OneDrive\Desktop\Suraj\Reinforment Learning\gridworld\venv\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [6]:
EPOCHS = 20000
ALPHA = 0.8
GAMMA = 0.9
NO_OF_BINS = 10

epsilon = 1.0
min_epsilon = 0.01
max_epsilon = 1.0
decay_val = 0.0045

In [7]:
def createBins(noOfBins):
    binCartPosition = np.linspace(-4.8, 4.8, noOfBins)
    binCartVelocity = np.linspace(-5, 5, noOfBins)
    binPoleAngle = np.linspace(-0.418, 0.418, noOfBins)
    binAngularVelocity = np.linspace(-5, 5, noOfBins)

    return np.array([binCartPosition, binCartVelocity, binPoleAngle, binAngularVelocity])

In [10]:
bins = createBins(NO_OF_BINS)

In [8]:
def discretizeObs(observation, bins):
    binned_obs = []
    for i, obs in enumerate(observation):
        discreetVal = np.digitize(obs, bins[i])
        binned_obs.append(discreetVal)

    return tuple(binned_obs)

In [9]:
observation

array([ 0.05350843, -0.03458495, -0.04756857, -0.06274901], dtype=float32)

In [11]:
discretizeObs(observation, bins)

(5, 5, 4, 5)

In [12]:
qTableShape = (NO_OF_BINS, NO_OF_BINS, NO_OF_BINS, NO_OF_BINS, env.action_space.n)

In [13]:
qTable = np.zeros(qTableShape)

In [15]:
qTable.shape

(10, 10, 10, 10, 2)

In [ ]:
for epoch in range(EPOCHS):
    initState = env.reset()